# NISQ Experiment

In [19]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from qiskit import Aer
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from qiskit.providers.aer import AerSimulator
from copy import deepcopy

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from utils import *
from experiments import *
from qiskit.providers.fake_provider import FakeCasablanca
#np.set_printoptions(threshold=sys.maxsize)

In [20]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.tools.visualization import circuit_drawer

In [21]:
#qk.IBMQ.save_account("66718f8f8aef22bcb6ebe86ad94a11f1fd1f4c55100829bb13f16e6b448e0a1ec6d09c459d738f58d0cbd8398a2a1f5e185a4706a61b6f896a5ce2983e136429", overwrite=True) 
provider = qk.IBMQ.load_account()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend("ibmq_manila")

#backend = AerSimulator()

ibmqfactory.load_account:WARNING:2022-08-10 11:50:00,158: Credentials are already in use. The existing account in the session will be replaced.


## Pauli String Expectation Values

## Two Qubit POVM

In [5]:
n = 2
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 324
input_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)

for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n)) 

      ┌────────────┐ ┌───┐┌─────────────┐     
q_0: ─┤ Ry(2.8319) ├─┤ X ├┤ Rx(0.61989) ├──■──
     ┌┴────────────┴┐└─┬─┘└┬────────────┤┌─┴─┐
q_1: ┤ Ry(-0.78829) ├──■───┤ Rx(1.4577) ├┤ X ├
     └──────────────┘      └────────────┘└───┘


In [6]:
job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
counts_list = [result1.get_counts(circuit) for circuit in circuit_list[:100]]
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])

data = [input_list, counts_list[:N], counts_list[N:N+20], counts_list[N+20:N+20+d**2]]

pickle.dump(data, open("../../data/twoQubits_pauliStrings_corrMat.p", "wb"))

### Three Qubits

In [22]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()
print(circuit_target)

N = 1000-20-d**2
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N)


for i in range(1, 21):
    q_reg = qk.QuantumRegister(n)
    c_reg = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(q_reg, c_reg)
    for j in range(i):
        circuit = circuit.compose(circuit_target)
        
    circuit.measure(q_reg, c_reg)
    circuit_list.append(circuit)
    
for i in range(d):
    circuit_list.extend(generate_bitstring_circuits(n)) 

      ┌────────────┐           ┌───┐     ┌─────────────┐          
q_0: ─┤ Ry(1.4577) ├───────────┤ X ├─────┤ Rx(-2.1615) ├──■───────
      ├────────────┤ ┌───┐     └─┬─┘     ├─────────────┤┌─┴─┐     
q_1: ─┤ Ry(2.8319) ├─┤ X ├───────■───────┤ Rx(-2.1613) ├┤ X ├──■──
     ┌┴────────────┴┐└─┬─┘┌─────────────┐└─────────────┘└───┘┌─┴─┐
q_2: ┤ Ry(-0.78829) ├──■──┤ Rx(0.61989) ├────────────────────┤ X ├
     └──────────────┘     └─────────────┘                    └───┘


In [23]:
job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job5 = qk.execute(circuit_list[400:500], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
result5 = job5.result()

counts_list = [result1.get_counts(circuit) for circuit in circuit_list[:100]]
counts_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])
counts_list.extend([result5.get_counts(circuit) for circuit in circuit_list[400:500]])

data = [input_map_list, counts_list[:N], counts_list[N:N+20], counts_list[N+20:N+20+d**2]]

pickle.dump(data, open("../../data/threeQubits_pauliStrings_corrMat.p", "wb"))

In [24]:
job6 = qk.execute(circuit_list[500:600], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job7 = qk.execute(circuit_list[600:700], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job8 = qk.execute(circuit_list[700:800], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job9 = qk.execute(circuit_list[800:900], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job10 = qk.execute(circuit_list[900:1000], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result6 = job6.result()
result7 = job7.result()
result8 = job8.result()
result9 = job9.result()
result10 = job10.result()

counts_list.extend([result6.get_counts(circuit) for circuit in circuit_list[500:600]])
counts_list.extend([result7.get_counts(circuit) for circuit in circuit_list[600:700]])
counts_list.extend([result8.get_counts(circuit) for circuit in circuit_list[700:800]])
counts_list.extend([result9.get_counts(circuit) for circuit in circuit_list[800:900]])
counts_list.extend([result10.get_counts(circuit) for circuit in circuit_list[900:1000]])

data = [input_map_list, counts_list[:N], counts_list[N:N+20], counts_list[N+20:N+20+d**2]]

pickle.dump(data, open("../../data/threeQubits_pauliStrings_corrMat_2.p", "wb"))

### More Data

In [30]:
input_map_list, counts_map_list, counts_rep_list, counts_corr_list = pickle.load(open("../../data/threeQubits_pauliStrings_corrMat_2.p", "rb"))
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)

circuit_target = variational_circuit(n).reverse_bits()

N = 1500-20-d**2
input_map_list, circuit_list = generate_pauli_circuits(n, circuit_target, N, trace=False)

circuit_list = circuit_list[-500:]

In [31]:
job1 = qk.execute(circuit_list[:100], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job2 = qk.execute(circuit_list[100:200], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job3 = qk.execute(circuit_list[200:300], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job4 = qk.execute(circuit_list[300:400], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)
job5 = qk.execute(circuit_list[400:500], backend, shots = 20000, optimization_level = 0, seed_transpiler=42)

result1 = job1.result()
result2 = job2.result()
result3 = job3.result()
result4 = job4.result()
result5 = job5.result()

counts_map_list.extend([result1.get_counts(circuit) for circuit in circuit_list[:100]])
counts_map_list.extend([result2.get_counts(circuit) for circuit in circuit_list[100:200]])
counts_map_list.extend([result3.get_counts(circuit) for circuit in circuit_list[200:300]])
counts_map_list.extend([result4.get_counts(circuit) for circuit in circuit_list[300:400]])
counts_map_list.extend([result5.get_counts(circuit) for circuit in circuit_list[400:500]])


data = [input_map_list, counts_map_list, counts_rep_list, counts_corr_list]

pickle.dump(data, open("../../data/threeQubits_pauliStrings_corrMat_3.p", "wb"))